In [1]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from io import StringIO
import os

### Celda para extraer info de tres tablas de la Euro Femenina

In [2]:
# === Configurar Selenium ===
driver_path = r"C:\Users\Ramón\AppData\Roaming\chromedriver-win64\chromedriver.exe"

options = Options()
# options.add_argument("--headless")  # Puedes activarlo si no quieres abrir el navegador
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service, options=options)

# === Funciones ===
def get_team_name_from_url(url):
    name = url.split("/")[-1].replace("-Women-Stats", "")
    return name.replace("-", " ")

def get_df_from_table(soup, table_prefix):
    pattern = re.compile(f"^{table_prefix}(_\\d+)?$")
    table = soup.find("table", id=pattern)
    if table:
        try:
            df = pd.read_html(StringIO(str(table)), header=[0, 1])[0]

            # Aplanar columnas jerárquicas
            df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

            # Corregir columnas con 'Unnamed'
            df = df.rename(columns=lambda col: re.sub(r"^Unnamed: \d+_level_0_", "", col))

            return df
        except Exception as e:
            print(f"⚠️ Error leyendo la tabla: {e}")
            return pd.DataFrame()
    return pd.DataFrame()


def scrape_team_data(url):
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    team_name = get_team_name_from_url(url)
    print(f"📥 Procesando equipo: {team_name}")

    df_std = get_df_from_table(soup, "stats_standard")
    df_pas = get_df_from_table(soup, "stats_passing")
    df_gca = get_df_from_table(soup, "stats_gca")

    if df_std.empty and df_pas.empty and df_gca.empty:
        print(f"❌ No se extrajeron datos para {team_name}")
        return None

    df_merged = df_std
    if not df_pas.empty:
        df_merged = df_merged.merge(df_pas, on="Player", how="outer", suffixes=('', '_passing'))
    if not df_gca.empty:
        df_merged = df_merged.merge(df_gca, on="Player", how="outer", suffixes=('', '_gca'))

    df_merged["Team"] = team_name
    return df_merged

# === Ejecutar scraping ===
urls = [
    "https://fbref.com/en/squads/46db89e7/Belgium-Women-Stats",
    "https://fbref.com/en/squads/a8855f01/Denmark-Women-Stats",
    "https://fbref.com/en/squads/5849ebe1/England-Women-Stats",
    "https://fbref.com/en/squads/97d1aa04/Spain-Women-Stats",
    "https://fbref.com/en/squads/8df8987f/Finland-Women-Stats",
    "https://fbref.com/en/squads/064c6283/France-Women-Stats",
    "https://fbref.com/en/squads/0a9f476d/Germany-Women-Stats",
    "https://fbref.com/en/squads/ecb5d7d5/Iceland-Women-Stats",
    "https://fbref.com/en/squads/379e8f43/Italy-Women-Stats",
    "https://fbref.com/en/squads/ec991a3d/Netherlands-Women-Stats",
    "https://fbref.com/en/squads/1ec54f37/Norway-Women-Stats",
    "https://fbref.com/en/squads/ee429419/Poland-Women-Stats",
    "https://fbref.com/en/squads/29332db8/Portugal-Women-Stats",
    "https://fbref.com/en/squads/4173add7/Sweden-Women-Stats",
    "https://fbref.com/en/squads/b08f950e/Switzerland-Women-Stats",
    "https://fbref.com/en/squads/f4adb447/Wales-Women-Stats"
]

all_teams_data = []

for url in urls:
    df_team = scrape_team_data(url)
    if df_team is not None:
        all_teams_data.append(df_team)

# Unir todos los equipos
df_all = pd.concat(all_teams_data, ignore_index=True)
driver.quit()

# Mostrar muestra
df_all.head()

📥 Procesando equipo: Belgium
📥 Procesando equipo: Denmark
📥 Procesando equipo: England
📥 Procesando equipo: Spain
📥 Procesando equipo: Finland
📥 Procesando equipo: France
📥 Procesando equipo: Germany
📥 Procesando equipo: Iceland
📥 Procesando equipo: Italy
📥 Procesando equipo: Netherlands
📥 Procesando equipo: Norway
📥 Procesando equipo: Poland
📥 Procesando equipo: Portugal
📥 Procesando equipo: Sweden
📥 Procesando equipo: Switzerland
📥 Procesando equipo: Wales


,Player,Pos,Age,MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,Performance_G+A,...,GCA_GCA,GCA_GCA90,GCA Types_PassLive,GCA Types_PassDead,GCA Types_TO,GCA Types_Sh,GCA Types_Fld,GCA Types_Def,Matches_gca,Team
0,Amber Tysiak,DF,25.0,3,3,270.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Matches,Belgium
1,Davina Philtjens,DF,35.0,1,0,12.0,0.1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Matches,Belgium
2,Elena Dhont,"FW,DF",26.0,3,0,55.0,0.6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Matches,Belgium
3,Ella Van Kerkhoven,FW,31.0,1,0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Matches,Belgium
4,Femke Bastiaen,GK,23.0,0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Belgium


### Comprobación df inicial

In [3]:
# Ver estructura general: columnas, tipos, nulos, memoria
df_all.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 81 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Player                   398 non-null    object 
 1   Pos                      366 non-null    object 
 2   Age                      396 non-null    float64
 3   MP                       398 non-null    int64  
 4   Playing Time_Starts      398 non-null    int64  
 5   Playing Time_Min         346 non-null    float64
 6   Playing Time_90s         346 non-null    float64
 7   Performance_Gls          346 non-null    float64
 8   Performance_Ast          346 non-null    float64
 9   Performance_G+A          346 non-null    float64
 10  Performance_G-PK         346 non-null    float64
 11  Performance_PK           346 non-null    float64
 12  Performance_PKatt        346 non-null    float64
 13  Performance_CrdY         346 non-null    float64
 14  Performance_CrdR         3

In [4]:
# Lista de todas las columnas
print(df_all.columns.tolist())


['Player', 'Pos', 'Age', 'MP', 'Playing Time_Starts', 'Playing Time_Min', 'Playing Time_90s', 'Performance_Gls', 'Performance_Ast', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR', 'Expected_xG', 'Expected_npxG', 'Expected_xAG', 'Expected_npxG+xAG', 'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR', 'Per 90 Minutes_Gls', 'Per 90 Minutes_Ast', 'Per 90 Minutes_G+A', 'Per 90 Minutes_G-PK', 'Per 90 Minutes_G+A-PK', 'Per 90 Minutes_xG', 'Per 90 Minutes_xAG', 'Per 90 Minutes_xG+xAG', 'Per 90 Minutes_npxG', 'Per 90 Minutes_npxG+xAG', 'Matches', 'Pos_passing', 'Age_passing', '90s', 'Total_Cmp', 'Total_Att', 'Total_Cmp%', 'Total_TotDist', 'Total_PrgDist', 'Short_Cmp', 'Short_Att', 'Short_Cmp%', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp%', 'Long_Cmp', 'Long_Att', 'Long_Cmp%', 'Ast', 'xAG', 'Expected_xA', 'Expected_A-xAG', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP', 'Matches_passing', 'Pos_gca', 'Age_gca', '90s_gca', 'SCA_SCA', 

### Renombrar, borrar y ordenar columnas

In [5]:
# Diccionario de columnas a conservar y su nuevo nombre
column_map = {
    'Player': 'Jugadora',
    'Pos': 'Posicion',
    'Age': 'Edad',
    'MP': 'Partidos disputados',
    'Playing Time_Min': 'Minutos',
    'Performance_Gls': 'Goles',
    'Performance_Ast': 'Asistencias',
    'Performance_G+A': 'Gls+Asist',
    'Performance_PK': 'Gol_penalti',
    'Performance_CrdY': 'Amarillas',
    'Performance_CrdR': 'Rojas',
    'Expected_xG': 'xG',
    'Expected_npxG': 'xG sin penal',
    'Expected_xAG': 'xAG',
    'Expected_npxG+xAG': 'xAG sin penal',
    'Progression_PrgC': 'Conducciones',
    'Progression_PrgP': 'Pases Progresivos',
    'Progression_PrgR': 'Pases progresivos recibidos',
    'Per 90 Minutes_Gls': 'Goles x90',
    'Per 90 Minutes_Ast': 'Asistencias x90',
    'Per 90 Minutes_G+A': 'Gls+Asist x90',
    'Per 90 Minutes_xG': 'xG x90',
    'Per 90 Minutes_xAG': 'xAG x90',
    'Per 90 Minutes_xG+xAG': 'xG+xAG x90',
    'Total_Cmp': 'Pases completados',
    'Total_Att': 'Pases intentados',
    'Total_Cmp%': '% pases',
    'Total_TotDist': 'Distancia total pases',
    'Total_PrgDist': 'Distancia pases progresivos',
    'Short_Cmp': 'Pases cortos exit',
    'Short_Att': 'Pases cortos intent',
    'Short_Cmp%': '% pases cortos',
    'Medium_Cmp': 'Pases medios exit',
    'Medium_Att': 'Pases medios intent',
    'Medium_Cmp%': '% pases medios',
    'Long_Cmp': 'Pases largos exit',
    'Long_Att': 'Pases largos intent',
    'Long_Cmp%': '% pases largos',
    'Expected_xA': 'xA',
    'Expected_A-xAG': 'A-xAG',
    'KP': 'Pases clave',
    '1/3': 'Pases ultimo tercio',
    'PPA': 'Pases area rival',
    'CrsPA': 'Centros al area',
    'SCA_SCA': 'Acciones crean tiro',
    'SCA_SCA90': 'Acciones crean tiro x90',
    'SCA Types_PassLive': 'Pase seguido de tiro',
    'SCA Types_PassDead': 'Pase ABP seguido de tiro',
    'SCA Types_Def': 'Acciones df seguidas de tiro',
    'GCA_GCA': 'Acciones generan gol',
    'GCA_GCA90': 'Acciones generan gol x90',
    'GCA Types_PassDead': 'Asistencia desde ABP',
    'GCA Types_Def': 'Acciones df generan gol',
    'Team': 'Pais'
}

# 1. Filtrar y renombrar
df_selected = df_all[list(column_map.keys())].copy()
df_selected.rename(columns=column_map, inplace=True)

# 2. Reordenar columnas: mover "Pais" detrás de "Jugadora"
cols = df_selected.columns.tolist()
cols.remove('Pais')
cols.insert(1, 'Pais')  # insertar después de 'Jugadora'
df_selected = df_selected[cols]

# 3. Mostrar resultado
display(df_selected.head())


,Jugadora,Pais,Posicion,Edad,Partidos disputados,Minutos,Goles,Asistencias,Gls+Asist,Gol_penalti,...,Centros al area,Acciones crean tiro,Acciones crean tiro x90,Pase seguido de tiro,Pase ABP seguido de tiro,Acciones df seguidas de tiro,Acciones generan gol,Acciones generan gol x90,Asistencia desde ABP,Acciones df generan gol
0,Amber Tysiak,Belgium,DF,25.0,3,270.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Davina Philtjens,Belgium,DF,35.0,1,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Elena Dhont,Belgium,"FW,DF",26.0,3,55.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ella Van Kerkhoven,Belgium,FW,31.0,1,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Femke Bastiaen,Belgium,GK,23.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Eliminar filas de totales de equipo y sus oponentes

In [6]:
# Eliminar filas con valores de resumen del equipo
df_selected = df_selected[~df_selected['Jugadora'].isin(['Squad Total', 'Opponent Total'])].reset_index(drop=True)

# Verificamos rápidamente
display(df_selected.head())


,Jugadora,Pais,Posicion,Edad,Partidos disputados,Minutos,Goles,Asistencias,Gls+Asist,Gol_penalti,...,Centros al area,Acciones crean tiro,Acciones crean tiro x90,Pase seguido de tiro,Pase ABP seguido de tiro,Acciones df seguidas de tiro,Acciones generan gol,Acciones generan gol x90,Asistencia desde ABP,Acciones df generan gol
0,Amber Tysiak,Belgium,DF,25.0,3,270.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Davina Philtjens,Belgium,DF,35.0,1,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Elena Dhont,Belgium,"FW,DF",26.0,3,55.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ella Van Kerkhoven,Belgium,FW,31.0,1,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Femke Bastiaen,Belgium,GK,23.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Quedarnos con la posición inicial en las celdas donde vengan dos

In [7]:
df_selected['Posicion'] = df_selected['Posicion'].astype(str).str.split(',').str[0]


### Renombrar paises

In [8]:
mapeo_paises = {
    'Belgium': 'Belgica',
    'Denmark': 'Dinamarca',
    'England': 'Inglaterra',
    'Spain': 'España',
    'Finland': 'Finlandia',
    'France': 'Francia',
    'Germany': 'Alemania',
    'Iceland': 'Islandia',
    'Italy': 'Italia',
    'Netherlands': 'Paises Bajos',
    'Norway': 'Noruega',
    'Poland': 'Polonia',
    'Portugal': 'Portugal',
    'Sweden': 'Suecia',
    'Switzerland': 'Suiza',
    'Wales': 'Gales'
}

df_selected['Pais'] = df_selected['Pais'].replace(mapeo_paises)


### Rellenamos edad de las jugadoras que no lo traen

In [9]:
df_selected.loc[df_selected['Jugadora'] == 'Heidi Kollanen', 'Edad'] = 28
df_selected.loc[df_selected['Jugadora'] == 'Olga Ahtinen', 'Edad'] = 28

In [21]:
df_eurowomen = df_selected.copy()

### Valores faltantes y nulos

In [22]:
# Detectar columnas numéricas a partir de "Minutos"
cols_metricas = df_selected.columns[5:]  # Desde 'Minutos' hasta el final

# Eliminar jugadoras con todos NaN en las métricas
df_eurowomen = df_selected.dropna(subset=cols_metricas, how='all')


In [23]:
# Rellenar valores faltantes con 0
df_eurowomen.fillna(0.0, inplace=True)

C:\Users\Ramón\AppData\Local\Temp\ipykernel_976\421325479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eurowomen.fillna(0.0, inplace=True)


In [24]:
# Verificar si aún hay valores faltantes
print(df_eurowomen.isnull().sum().sum())  # Esto debe devolver 0

0


In [ ]:
df_eurowomen.reset_index(drop=True, inplace=True)

### Pasar a int los float que queramos

In [27]:
# Lista de columnas que deseas convertir a int
cols_to_int = [
    'Edad', 'Minutos', 'Goles', 'Asistencias', 'Gls+Asist', 'Gol_penalti',
    'Amarillas', 'Rojas', 'Pases completados', 'Pases intentados',
    'Pases cortos exit', 'Pases cortos intent', 'Pases medios exit', 'Pases medios intent',
    'Pases largos exit', 'Pases largos intent', 'Pases clave',
    'Pases ultimo tercio', 'Pases area rival', 'Centros al area', 'Distancia total pases','Distancia pases progresivos',
    'Conducciones', 'Pases Progresivos', 'Pases progresivos recibidos',    
    'Acciones crean tiro', 'Pase seguido de tiro', 'Pase ABP seguido de tiro',
    'Acciones df seguidas de tiro', 'Acciones generan gol', 'Asistencia desde ABP',
    'Acciones df generan gol'
]

# Convertir a int sin errores (asegura que no hay NaNs)
df_eurowomen[cols_to_int] = df_eurowomen[cols_to_int].astype(int)


C:\Users\Ramón\AppData\Local\Temp\ipykernel_976\3955083621.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eurowomen[cols_to_int] = df_eurowomen[cols_to_int].astype(int)


In [28]:
df_eurowomen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Jugadora                      314 non-null    object 
 1   Pais                          314 non-null    object 
 2   Posicion                      314 non-null    object 
 3   Edad                          314 non-null    int64  
 4   Partidos disputados           314 non-null    int64  
 5   Minutos                       314 non-null    int64  
 6   Goles                         314 non-null    int64  
 7   Asistencias                   314 non-null    int64  
 8   Gls+Asist                     314 non-null    int64  
 9   Gol_penalti                   314 non-null    int64  
 10  Amarillas                     314 non-null    int64  
 11  Rojas                         314 non-null    int64  
 12  xG                            314 non-null    float64
 13  xG si

In [29]:
df_eurowomen.head()

,Jugadora,Pais,Posicion,Edad,Partidos disputados,Minutos,Goles,Asistencias,Gls+Asist,Gol_penalti,...,Centros al area,Acciones crean tiro,Acciones crean tiro x90,Pase seguido de tiro,Pase ABP seguido de tiro,Acciones df seguidas de tiro,Acciones generan gol,Acciones generan gol x90,Asistencia desde ABP,Acciones df generan gol
0,Amber Tysiak,Belgica,DF,25,3,270,0,0,0,0,...,0,0,0.00,0,0,0,0,0.00,0,0
1,Davina Philtjens,Belgica,DF,35,1,12,0,0,0,0,...,0,0,0.00,0,0,0,0,0.00,0,0
2,Elena Dhont,Belgica,FW,26,3,55,0,0,0,0,...,0,1,1.64,1,0,0,0,0.00,0,0
3,Ella Van Kerkhoven,Belgica,FW,31,1,4,0,0,0,0,...,0,0,0.00,0,0,0,0,0.00,0,0
4,Hannah Eurlings,Belgica,FW,22,2,137,1,0,1,0,...,0,6,3.94,5,1,0,1,0.66,1,0


In [30]:
# Verificar si aún hay valores faltantes
print(df_eurowomen.isnull().sum().sum())  # Esto debe devolver 0

0


In [32]:
df_eurowomen.shape  # Verificar la forma del DataFrame

(314, 54)

### Mejorar textos sin tildes, caracteres...

In [34]:
import unicodedata

def quitar_tildes(texto):
    if isinstance(texto, str):
        return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto


In [35]:
for col in df_eurowomen.select_dtypes(include='object').columns:
    df_eurowomen[col] = df_eurowomen[col].apply(quitar_tildes)


C:\Users\Ramón\AppData\Local\Temp\ipykernel_976\3374163331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eurowomen[col] = df_eurowomen[col].apply(quitar_tildes)


### Pasar a csv

In [36]:
# Guardar el DataFrame como CSV en la ruta deseada
df_eurowomen.to_csv("../data/euro_women/euro_women.csv", index=False)
